# Install Repository

In [ ]:
#first time run
!git clone https://github.com/antonioverdi/MLReproChallenge.git
#At the moment all requirements are met by Google Colab already I believe
import os
os.chdir("MLReprochallenge")
!pip install -r requirements.txt

In [ ]:
#subsequent runs
!git pull
!pip install -r requirements.txt

# Train Models
### Main.py Arguments
<table>
    <tr><th>Argument</th><th>Default</th><th>help</th></tr>
    <tr><td>--arch, - a</td><td>str: 'resnet56'</td><td>model architecture</td></tr>
    <tr><td>-j, --workers</td><td>int: 4</td><td>number of data loading workers</td></tr>
    <tr><td>--epochs</td><td>int: 182</td><td>number of total epochs to run</td></tr>
    <tr><td>--start-epoch</td><td>int: 0</td><td>manual epoch number, for restarts</td></tr>
    <tr><td>-b, --batch-size</td><td>int: 128</td><td>mini-batch size</td></tr>
    <tr><td>--lr, --learning-rate</td><td>float: 0.1</td><td>initial learning rate</td></tr>
    <tr><td>--momentum</td><td>float: 0.9</td><td>momentum</td></tr>
    <tr><td>--weight-decay, --wd</td><td>float: 2e-4</td><td>weight decay</td></tr>
    <tr><td>--print-freq, -p</td><td>int: 50</td><td>print frequency</td></tr>
    <tr><td>--resume</td><td>str: ' '</td><td>path to latest checkpoint</td></tr>
    <tr><td>-e, --evaluate</td><td>bool: False</td><td>evaluate model on validation set</td></tr>
    <tr><td>--pretrained</td><td>bool: False</td><td>use pre-trained model</td></tr>
    <tr><td>--half</td><td>bool: False</td><td>use half-precision float(16-bit)</td></tr>
    <tr><td>--save-dir</td><td>str: 'save_temp'</td><td>Directory used to save trained models</td></tr>
    <tr><td>--save-every</td><td>int: 10</td><td>Save checkpoint at every specified number of epochs</td></tr>
    <tr><td>--colab</td><td>bool: False</td><td>Set this to true when running in Google Colab</td></tr>
   </table>

In [10]:
%%time
!python main.py --arch="resnet32_l0"

L0Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), droprate_init=0.5, temperature=0.6666666666666666, prior_prec=1.4285714285714286, lamba=1.0, local_rep=False, padding=(1, 1), bias=False)
MAPConv2d(16, 16, kernel_size=(3, 3) , stride=(1, 1), weight_decay=1, padding=(1, 1), bias=False)
L0Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), droprate_init=0.5, temperature=0.6666666666666666, prior_prec=1.4285714285714286, lamba=1.0, local_rep=False, padding=(1, 1), bias=False)
MAPConv2d(16, 16, kernel_size=(3, 3) , stride=(1, 1), weight_decay=1, padding=(1, 1), bias=False)
L0Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), droprate_init=0.5, temperature=0.6666666666666666, prior_prec=1.4285714285714286, lamba=1.0, local_rep=False, padding=(1, 1), bias=False)
MAPConv2d(16, 16, kernel_size=(3, 3) , stride=(1, 1), weight_decay=1, padding=(1, 1), bias=False)
L0Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), droprate_init=0.5, temperature=0.6666666666666666, prior_prec=1.4285714285714286, la

C:\Users\juggl\OneDrive\Desktop\Concordia\COMP 432\Project\MLReproChallenge\l0_layers.py:191: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.weights, mode='fan_in')
C:\Users\juggl\OneDrive\Desktop\Concordia\COMP 432\Project\MLReproChallenge\base_layers.py:93: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(self.weight, mode='fan_in')
Traceback (most recent call last):
  File "main.py", line 312, in <module>
    main()
  File "main.py", line 70, in main
    model.cuda()
  File "C:\Toolkits\anaconda3-5.2.0\lib\site-packages\torch\nn\modules\module.py", line 463, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "C:\Toolkits\anaconda3-5.2.0\lib\site-packages\torch\nn\modules\module.py", line 359, in _apply
    module._apply(fn)
  File "C:\Toolkits\anaconda3-5.2.0\lib\site-packages\torch\nn\modules\module.py", line 359, in _apply
    module._

# Test Models